In [1]:
import pandas as pd
df_clean = pd.read_csv('./credit_data_clean.csv')

In [2]:
df_clean

,Age,Credit amount,Duration,Sex_female,Sex_male,Housing_free,Housing_own,Housing_rent,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Savings_Acc,Checking_Acc,Job_Skill,Bad_Risk
0,-1.094447,0.913171,2.139356,True,False,False,True,False,False,False,False,False,False,True,False,False,0,1,2,1
1,0.858575,1.573037,1.657897,False,True,True,False,False,False,False,False,False,True,False,False,False,0,0,2,0
2,1.537887,0.543768,0.213521,False,True,True,False,False,False,True,False,False,False,False,False,False,0,0,2,1
3,0.009435,1.253868,1.176438,False,True,False,False,True,False,True,False,False,False,False,False,False,0,1,3,0
4,-0.584963,0.668155,0.694979,False,True,False,True,False,False,True,False,False,False,False,False,False,0,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,1.113317,-0.524799,0.213521,False,True,False,True,False,False,False,False,False,False,True,False,False,0,1,1,0
518,-0.415135,0.232459,1.176438,False,True,False,True,False,False,False,False,False,True,False,False,False,0,0,3,0
519,0.434005,0.197603,0.694979,False,True,False,True,False,False,True,False,False,False,False,False,False,0,0,3,0
520,-1.009533,-0.489943,1.898626,False,True,True,False,False,False,False,False,False,False,True,False,False,0,0,2,1


In [3]:
from sklearn.model_selection import train_test_split
X = df_clean.drop(columns='Bad_Risk')
y= df_clean['Bad_Risk']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [4]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((417, 19), (105, 19), (417,), (105,))

In [5]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

C:\Users\victo\AppData\Local\Temp\ipykernel_12300\85489818.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [6]:
def build_model(hp):
    model = Sequential([
        Dense(
            units=hp.Int('units1', min_value=16, max_value=64, step=16),
            activation='relu',
            input_shape=(X_train.shape[1],),
            kernel_regularizer=l2(hp.Float('l2_1',min_value=0.001, max_value=0.01, sampling='LOG'))
        ),
        Dropout(hp.Float('dropout1', min_value=0.1, max_value=0.5, step=0.1)),
        Dense(
            units=hp.Int('units2', min_value=16, max_value=64, step=16),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_2', min_value=0.001, max_value=0.01, sampling='LOG'))
        ),
        Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)),
        Dense(
            units=hp.Int('units3', min_value=8, max_value=32, step=8),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_3', min_value=0.001, max_value=0.01, sampling='LOG'))
        ),
        Dropout(hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='keras_tuner',
    project_name='model_opt_2'
)

tuner.search(X_train, y_train, epochs=20, validation_split=0.2)

Trial 8 Complete [00h 00m 05s]
val_accuracy: 0.6309523582458496

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 39s

Search: Running Trial #9

Value             |Best Value So Far |Hyperparameter
16                |32                |units1
0.001698          |0.0091661         |l2_1
0.4               |0.4               |dropout1
16                |48                |units2
0.0021183         |0.0046902         |l2_2
0.3               |0.4               |dropout_2
32                |16                |units3
0.0014217         |0.0065752         |l2_3
0.5               |0.5               |dropout_3
0.0058977         |0.0017139         |learning_rate

Epoch 1/20


C:\Users\victo\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.5039 - loss: 0.8603 - val_accuracy: 0.5000 - val_loss: 0.7924
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5859 - loss: 0.7812 - val_accuracy: 0.5476 - val_loss: 0.7659
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6064 - loss: 0.7643 - val_accuracy: 0.5714 - val_loss: 0.7502
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5670 - loss: 0.7523 - val_accuracy: 0.5714 - val_loss: 0.7424
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6748 - loss: 0.6957 - val_accuracy: 0.6190 - val_loss: 0.7261
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6731 - loss: 0.6840 - val_accuracy: 0.6190 - val_loss: 0.7251
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6828 - loss: 0.6808 - val_accuracy: 0.6190 - val_loss: 0.7229
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6297 - loss: 0.6900 - val_accuracy: 0.6190 - val_loss: 0.7346
Ep

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

In [ ]:
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")